## Koster Lab Statistics

In [28]:
from utils import db_utils, zooniverse_utils
import pandas as pd
import io, json

In [2]:
# Set the path for the db
db_path = "/data/database/demo.db"

# Specify Google Drive ids of the csv files with information about the species choices,
# original movies and duplicated clips
sp_file_id = "1dnueH3BjJrMK8buVjfyFbxfu0E-5dX7Z"
mv_file_id = "1LL-Ah_FIkBiGKEldYvuhNeL2NyOvKBip"
dp_file_id = "1z72CqTtEBtqk6936H1YNrCjc5NRopF0g"

# Specify username and password of a valid zooniverse account
user_zoo = "user"
pass_zoo = "pass"

# Species choice
species = 'DEEPWATERCORAL'

In [5]:
# Specify the Zooniverse workflows of interest and their versions
workflow_clip = 11767
workflow_clip_version = 227
workflow_frame = 12852
workflow_frame_version = 21.85

# Specify the agreement threshold required among cit scientists
agg_user_clip = 0.8
agg_user_frames = 0.8

# Specifiy the min number of different Zooniverse users required per subject
min_users_clip = 3
min_users_frames = 5

# Frame user agreement threshold
iua = 0.8

### Connect to Koster DB and retrieve relevant tables

In [6]:
conn = db_utils.create_connection("/data/database/demo.db")

In [7]:
subjects = pd.read_sql_query("SELECT * FROM subjects", conn)
movies = pd.read_sql_query("SELECT * FROM movies", conn)
sites = pd.read_sql_query("SELECT * FROM sites", conn)

### Create Clips Aggregate Table based on thresholds

In [8]:
# Connect to the Zooniverse project
project = zooniverse_utils.auth_session(user_zoo, pass_zoo)

# Get the classifications from the project
export = project.get_export("classifications")

# Save the response as pandas data frame
class_df = pd.read_csv(
    io.StringIO(export.content.decode("utf-8")),
    usecols=[
        "subject_ids",
        "classification_id",
        "workflow_id",
        "workflow_version",
        "annotations",
    ],
)


# Filter clip classifications
class_df = class_df[
    (class_df.workflow_id >= workflow_clip)
    & (class_df.workflow_version >= workflow_clip_version)
].reset_index()

# Drop worflow columns
class_df = class_df.drop(columns=["workflow_id", "workflow_version"])

# Create an empty list
rows_list = []

# Loop through each classification submitted by the users
for index, row in class_df.iterrows():
    # Load annotations as json format
    annotations = json.loads(row["annotations"])

    # Select the information from the species identification task
    for ann_i in annotations:
        if ann_i["task"] == "T4":

            # Select each species annotated and flatten the relevant answers
            for value_i in ann_i["value"]:
                choice_i = {}
                # If choice = 'nothing here', set follow-up answers to blank
                if value_i["choice"] == "NOTHINGHERE":
                    f_time = ""
                    inds = ""
                # If choice = species, flatten follow-up answers
                else:
                    answers = value_i["answers"]
                    for k in answers.keys():
                        if "FIRSTTIME" in k:
                            f_time = answers[k].replace("S", "")
                        if "INDIVIDUAL" in k:
                            inds = answers[k]

                # Save the species of choice, class and subject id
                choice_i.update(
                    {
                        "classification_id": row["classification_id"],
                        "label": value_i["choice"],
                        "first_seen": f_time,
                        "how_many": inds,
                    }
                )

                rows_list.append(choice_i)

# Create a data frame with annotations as rows
annot_df = pd.DataFrame(
    rows_list, columns=["classification_id", "label", "first_seen", "how_many"]
)

# Specify the type of columns of the df
annot_df["how_many"] = pd.to_numeric(annot_df["how_many"])
annot_df["first_seen"] = pd.to_numeric(annot_df["first_seen"])

# Add subject id to each annotation
annot_df = pd.merge(
    annot_df,
    class_df.drop(columns=["annotations"]),
    how="left",
    on="classification_id",
)

# Clear duplicated subjects
if dp_file_id:
    annot_df = db_utils.combine_duplicates(annot_df, dp_file_id)

# Calculate the number of users that classified each subject
annot_df["n_users"] = annot_df.groupby("subject_ids")[
    "classification_id"
].transform("nunique")

# Select subjects with at least n different user classifications
annot_df = annot_df[annot_df.n_users >= min_users_clip]

# Calculate the proportion of users that agreed on their annotations
annot_df["class_n"] = annot_df.groupby(["subject_ids", "label"])[
    "classification_id"
].transform("count")

annot_df["class_prop"] = annot_df.class_n / annot_df.n_users



### Clip classification statistics

In [9]:
print(f"Number of users classifying clips ranged from {annot_df.n_users.min()} to {annot_df.n_users.max()}")
print(f"Median number of users classifying clips {annot_df.n_users.median()}")

Number of users classifying clips ranged from 4 to 32
Median number of users classifying clips 9.0


In [10]:
print(f"Proportion of agreement in classifying clips ranged from {annot_df.class_prop.min()} to {annot_df.class_prop.max()}")
print(f"Median agreement prop of users classifying clips {annot_df.class_prop.median()}")

Proportion of agreement in classifying clips ranged from 0.03125 to 1.0
Median agreement prop of users classifying clips 0.4444444444444444


### Add movie and site information to subjects

In [11]:
annot_df = pd.merge(annot_df, subjects, left_on='subject_ids', right_on='id')
annot_df = pd.merge(annot_df, movies, left_on="movie_id", right_on="id")
annot_df = pd.merge(annot_df, sites, left_on="site_id", right_on="id")

In [12]:
def get_annotations(df, agg_thresh):
    return annot_df[(annot_df.label == species) & (annot_df.class_prop >= agg_thresh)].sort_values(
        by=["class_prop"], ascending=False)[["filename_x", "fpath", "name",
                                             "first_seen", "how_many", "label", "class_prop"]]

In [13]:
print(f"Number of clips for species {species}: {len(get_annotations(annot_df, 0.0).filename_x.value_counts())}")

Number of clips for species DEEPWATERCORAL: 1945


In [14]:
print(f"Number of clips for species with agreement {agg_user_clip} for {species}: \
      {len(get_annotations(annot_df, 0.8).filename_x.value_counts())}")

Number of clips for species with agreement 0.8 for DEEPWATERCORAL:       194


In [15]:
print(f"Number of clips for species with agreement {0.5} for {species}: \
      {len(get_annotations(annot_df, 0.5).filename_x.value_counts())}")

Number of clips for species with agreement 0.5 for DEEPWATERCORAL:       733


## Frame statistics

In [16]:
import os, io, csv, json
import requests, argparse
import pandas as pd
import numpy as np
from ast import literal_eval
from datetime import datetime
from panoptes_client import Project, Panoptes
from collections import OrderedDict, Counter
from sklearn.cluster import DBSCAN

In [17]:
def bb_iou(boxA, boxB):

    # Compute edges
    temp_boxA = boxA.copy()
    temp_boxB = boxB.copy()
    temp_boxA[2], temp_boxA[3] = (
        temp_boxA[0] + temp_boxA[2],
        temp_boxA[1] + temp_boxA[3],
    )
    temp_boxB[2], temp_boxB[3] = (
        temp_boxB[0] + temp_boxB[2],
        temp_boxB[1] + temp_boxB[3],
    )

    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(temp_boxA[0], temp_boxB[0])
    yA = max(temp_boxA[1], temp_boxB[1])
    xB = min(temp_boxA[2], temp_boxB[2])
    yB = min(temp_boxA[3], temp_boxB[3])

    # compute the area of intersection rectangle
    interArea = abs(max((xB - xA, 0)) * max((yB - yA), 0))
    if interArea == 0:
        return 1
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = abs((temp_boxA[2] - temp_boxA[0]) * (temp_boxA[3] - temp_boxA[1]))
    boxBArea = abs((temp_boxB[2] - temp_boxB[0]) * (temp_boxB[3] - temp_boxB[1]))

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the intersection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return 1 - iou


def filter_bboxes(total_users, users, bboxes, obj, eps, iua):

    # If at least half of those who saw this frame decided that there was an object
    user_count = pd.Series(users).nunique()
    if user_count / total_users >= obj:
        # Get clusters of annotation boxes based on iou criterion
        cluster_ids = DBSCAN(min_samples=1, metric=bb_iou, eps=eps).fit_predict(bboxes)
        # Count the number of users within each cluster
        counter_dict = Counter(cluster_ids)
        # Accept a cluster assignment if at least 80% of users agree on annotation
        passing_ids = [k for k, v in counter_dict.items() if v / user_count >= iua]

        indices = np.isin(cluster_ids, passing_ids)

        final_boxes = []
        for i in passing_ids:
            # Compute median over all accepted bounding boxes
            boxes = np.median(np.array(bboxes)[np.where(cluster_ids == i)], axis=0)
            final_boxes.append(boxes)

        return indices, final_boxes

    else:
        return [], bboxes

In [18]:
project = zooniverse_utils.auth_session(user_zoo, pass_zoo)

# Get the export classifications
export = project.get_export("classifications")

# Save the response as pandas data frame
rawdata = pd.read_csv(
    io.StringIO(export.content.decode("utf-8")),
    usecols=[
        "user_name",
        "subject_ids",
        "subject_data",
        "classification_id",
        "workflow_id",
        "workflow_version",
        "annotations",
    ],
)

# Filter w2 classifications
w2_data = rawdata[
    (rawdata.workflow_id >= workflow_frame)
    & (rawdata.workflow_version >= workflow_frame_version)
].reset_index()

# Clear duplicated subjects
if dp_file_id:
    w2_data = db_utils.combine_duplicates(w2_data, dp_file_id)

# Calculate the number of users that classified each subject
w2_data["n_users"] = w2_data.groupby("subject_ids")[
    "classification_id"
].transform("nunique")

In [19]:
print(f"Number of frames for species {species}: {len(w2_data.groupby('subject_ids'))}")

print(f"Number of users classifying clips ranged from {w2_data.n_users.min()} to {w2_data.n_users.max()}")
print(f"Median number of users classifying clips {w2_data.n_users.median()}")

Number of frames for species DEEPWATERCORAL: 533
Number of users classifying clips ranged from 5 to 12
Median number of users classifying clips 5.0


In [20]:
# Select frames with at least n different user classifications
w2_data = w2_data[w2_data.n_users >= min_users_frames]

# Drop workflow and n_users columns
w2_data = w2_data.drop(columns=["workflow_id", "workflow_version", "n_users"])

# Extract the video filename and annotation details
w2_data[["filename", "frame_number", "label"]] = pd.DataFrame(
    w2_data["subject_data"]
    .apply(
        lambda x: [
            {
                "filename": v["movie_filepath"],
                "frame_number": v["frame_number"],
                "label": v["label"],
            }
            for k, v in json.loads(x).items()
        ][0]
    )
    .tolist()
)

# Convert to dictionary entries
w2_data["filename"] = w2_data["filename"].apply(lambda x: {"filename": x})
w2_data["frame_number"] = w2_data["frame_number"].apply(
    lambda x: {"frame_number": x}
)
w2_data["label"] = w2_data["label"].apply(lambda x: {"label": x})
w2_data["user_name"] = w2_data["user_name"].apply(lambda x: {"user_name": x})
w2_data["subject_id"] = w2_data["subject_ids"].apply(lambda x: {"subject_id": x})
w2_data["annotation"] = w2_data["annotations"].apply(
    lambda x: literal_eval(x)[0]["value"], 1
)

# Extract annotation metadata
w2_data["annotation"] = w2_data[
    ["filename", "frame_number", "label", "annotation", "user_name", "subject_id"]
].apply(
    lambda x: [
        OrderedDict(
            list(x["filename"].items())
            + list(x["frame_number"].items())
            + list(x["label"].items())
            + list(x["annotation"][i].items())
            + list(x["user_name"].items())
            + list(x["subject_id"].items())
        )
        for i in range(len(x["annotation"]))
    ]
    if len(x["annotation"]) > 0
    else [
        OrderedDict(
            list(x["filename"].items())
            + list(x["frame_number"].items())
            + list(x["label"].items())
            + list(x["user_name"].items())
            + list(x["subject_id"].items())
        )
    ],
    1,
)

# Convert annotation to format which the tracker expects
ds = [
    OrderedDict(
        {
            "user": i["user_name"],
            "filename": i["filename"],
            "class_name": i["label"],
            "start_frame": i["frame_number"],
            "x": int(i["x"]) if "x" in i else None,
            "y": int(i["y"]) if "y" in i else None,
            "w": int(i["width"]) if "width" in i else None,
            "h": int(i["height"]) if "height" in i else None,
            "subject_id": i["subject_id"] if "subject_id" in i else None,
        }
    )
    for i in w2_data.annotation.explode()
    if i is not None and i is not np.nan
]

# Get prepared annotations
w2_full = pd.DataFrame(ds)
w2_annotations = w2_full[w2_full["x"].notnull()]
new_rows = []
final_indices = []
for name, group in w2_annotations.groupby(
    ["filename", "class_name", "start_frame"]
):

    filename, class_name, start_frame = name

    total_users = w2_full[
        (w2_full.filename == filename)
        & (w2_full.class_name == class_name)
        & (w2_full.start_frame == start_frame)
    ]["user"].nunique()

    # Filter bboxes using IOU metric (essentially a consensus metric)
    # Keep only bboxes where mean overlap exceeds this threshold
    indices, new_group = filter_bboxes(
        total_users=total_users,
        users=[i[0] for i in group.values],
        bboxes=[np.array((i[4], i[5], i[6], i[7])) for i in group.values],
        obj=0.8,
        eps=0.5,
        iua=iua,
    )

    subject_ids = [i[8] for i in group.values[indices]]

    for ix, box in zip(subject_ids, new_group):
        new_rows.append((filename, class_name, start_frame, ix,) + tuple(box))

w2_annotations = pd.DataFrame(
    new_rows,
    columns=[
        "filename",
        "class_name",
        "start_frame",
        "subject_id",
        "x",
        "y",
        "w",
        "h",
    ],
)


# Get species id for each species
conn = db_utils.create_connection(db_path)

# Get subject table
subjects_df = pd.read_sql_query(
    "SELECT id, frame_exp_sp_id, movie_id FROM subjects", conn
)
subjects_df = subjects_df.rename(
    columns={"id": "subject_id", "frame_exp_sp_id": "species_id"}
)

w2_annotations = pd.merge(
    w2_annotations,
    subjects_df,
    how="left",
    left_on="subject_id",
    right_on="subject_id",
    validate="many_to_one",
)

# Filter out invalid movies
w2_annotations = w2_annotations[w2_annotations.movie_id.notnull()][
    ["species_id", "class_name", "x", "y", "w", "h", "subject_id", "movie_id"]
]

In [21]:
# Filter by species
w2_annotations['class_name'] = w2_annotations.class_name.apply(lambda x: x.upper().replace(" ", ""), 1)
w2_annotations = w2_annotations[w2_annotations.class_name == species]

In [22]:
# Merge with movies and site to see distribution of instances
w2_annotations = pd.merge(w2_annotations, movies, left_on='movie_id', right_on='id')
w2_annotations = pd.merge(w2_annotations, sites, left_on='site_id', right_on='id')

In [23]:
print(f"Number of movies for species {species}: {len(w2_annotations.fpath.value_counts())}")
print(f"Number of subjects for species {species}: {len(w2_annotations.groupby('subject_id'))}")
print(f"Number of instances for species {species}: {len(w2_annotations)}")

Number of movies for species DEEPWATERCORAL: 17
Number of subjects for species DEEPWATERCORAL: 409
Number of instances for species DEEPWATERCORAL: 557


In [24]:
print(f"Site distribution {species}: \n\n{w2_annotations['name'].value_counts()}")

Site distribution DEEPWATERCORAL: 

missing                                  325
Revet, sacken                            149
sackenrevet alfa                          45
Revet, Sacken                             37
o. sundet Storo - Torso,utsidan ranna      1
Name: name, dtype: int64


In [25]:
print(f"Movie distribution {species}: \n\n{w2_annotations['fpath'].value_counts()}")

Movie distribution DEEPWATERCORAL: 

/uploads/000203 TMBL-ROV 2000 Säckenrevet Tape 56.mov                   107
/uploads/000203 TMBL-ROV 2000 Säcken revet EJ numrerade band.mov         81
/uploads/000114 TMBL-ROV 2000 Säckenrevet Tape 55.mov                    67
/uploads/010424 Säckenrevet alfa Tape 74.mov                             45
/uploads/010424 Säckenrevet beta Tape 74.mov                             38
/uploads/990506 TMBL-ROV 1999 Revet Säcken 2 Tape 42.mov                 37
/uploads/040522 TMBL-ROV 2004 Säckenrevet.mov                            36
/uploads/030317-18 TMBL-ROV 2003 Säckenrevet.mov                         33
/uploads/030325 TMBL-ROV 2003 Säckenrevet bioerosion.mov                 32
/uploads/000114 TMBL-ROV 2000 Säckenrevet EJ numrerade band.mov          27
/uploads/040220 TMBL-ROV 2004 Säckenrevet alfa.mov                       25
/uploads/000203 TMBL-ROV 2000 Säcken EJ numrerade band.mov               10
/uploads/990506 TMBL-ROV 1999 Revet Säcken Tape 42_

## Visualise species distribution

In [27]:
# Remove missing locations
w2_annotations["coord_lon"] = w2_annotations['coord_lon'].astype(float)
w2_annotations["coord_lat"] = w2_annotations['coord_lat'].astype(float)
w2_annotations = w2_annotations[(w2_annotations.coord_lon != 0.0) & (w2_annotations.coord_lat != 0.0)]

In [51]:
w2_counts = w2_annotations.groupby(["coord_lat", "coord_lon"]).size().reset_index(name='counts')

In [75]:
import folium
import datetime as dt
import random as rnd

#GBG Coordinates
GBG_COORD = [57.7089, 11.9746]

coords = [(x,y) for x,y in zip(w2_counts.coord_lat, w2_counts.coord_lon)]

# Build map
print(f'Species Distribution for {species} in Koster Protected Region')
map_gbg = folium.Map(location=GBG_COORD, zoom_start=6, 
tiles='Stamen Terrain', width=1000, height=500)

# Plot coordinates using comprehension list
[folium.CircleMarker(coords[i], radius=int(w2_counts['counts'][i]//10),
                color='#0080bb', popup=int(w2_counts['counts'][i]), fill_color='#0080bb').add_to(map_gbg) 
for i in range(len(coords))]

# Display map in Jupyter
map_gbg

Species Distribution for DEEPWATERCORAL in Koster Protected Region


In [76]:
#End of notebook